# Лабораторна робота №1
Тема: Підготовка статистичних даних: читання, описова статистика, очистка  
Студент: Луцюк Богдан КН-2  
Варіант: 4  
Дата: 06.10.2025  
Репозиторій: https://github.com/emissuu/UniTasks/tree/main/R/Lab1

### Parameters

In [2]:
library(dplyr)

n <- 500
seed <- 16


Attaching package: ‘dplyr’

The following objects are masked from ‘package:stats’:

    filter, lag

The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union



Warning message:
package ‘dplyr’ was built under R version 4.5.1 


### Section 1

Генеруємо дані

In [ ]:
set.seed(seed)

order_id       <- 1:n
created_at     <- as.POSIXct(
  runif(n, 
    as.numeric(c.POSIXct("2024-04-01 00:00:00")), 
    as.numeric(c.POSIXct("2024-04-30 23:59:59"))
  ),
origin = "1970-01-01", tz = "Europe/Kyiv"
)
status         <- sample(c("created", "assigned", "picked", "delivered", "canceled"), n, TRUE)
for (i in seq_len(n)) {
  if (status[i] == "created") {
    next
  } else if (status[i] == "assigned") {
    assigned_at[i] <- created_at[i] + sample(10:60, n, TRUE) * 60
  } else if (status[i] == "picked"){
  assigned_at <- created_at + sample(10:60, n, TRUE) * 60
  picked_at <- assigned_at + sample(10:60, n, TRUE) * 60
  } else if (status[i] == "delivered") {
    assigned_at <- created_at + sample(10:60, n, TRUE) * 60
    picked_at <- assigned_at + sample(10:60, n, TRUE) * 60
    delivered_at <- picked_at + sample(10:60, n, TRUE) * 60
  } else if
}
assigned_at    <- created_at + sample(10:60, n, TRUE) * 60
picked_at      <- assigned_at + sample(10:60, n, TRUE) * 60
delivered_at   <- picked_at + sample(10:60, n, TRUE) * 60


data <- data.frame(
  order_id         = order_id,
  created_at       = created_at,
  assigned_at      = assigned_at,
  picked_at        = picked_at,
  delivered_at     = delivered_at,
  status           = status,
  # customer_id      =
  # city             =
  # address          =
  # lat              =
  # lon              =
  # distance_km      =
  # service_level    =
  # payment_method   =
  # items_count      =
  # order_value      =
  # delivery_fee     =
  # courier_id       =
  # note             =
  stringsAsFactors = FALSE
)
data

    order_id          created_at         assigned_at           picked_at        delivered_at    status
1          1 2024-04-21 11:50:20 2024-04-21 12:13:20 2024-04-21 13:12:20 2024-04-21 14:08:20 delivered
2          2 2024-04-08 07:45:52 2024-04-08 08:25:52 2024-04-08 09:04:52 2024-04-08 09:23:52   created
3          3 2024-04-14 12:04:48 2024-04-14 12:56:48 2024-04-14 13:48:48 2024-04-14 14:06:48   created
4          4 2024-04-07 21:11:35 2024-04-07 21:38:35 2024-04-07 22:12:35 2024-04-07 22:50:35    picked
5          5 2024-04-26 21:43:31 2024-04-26 22:13:31 2024-04-26 22:56:31 2024-04-26 23:26:31  assigned
6          6 2024-04-10 08:03:50 2024-04-10 08:46:50 2024-04-10 09:20:50 2024-04-10 10:03:50  assigned
7          7 2024-04-03 05:31:22 2024-04-03 06:00:22 2024-04-03 06:41:22 2024-04-03 07:11:22   created
8          8 2024-04-25 23:16:16 2024-04-26 00:10:16 2024-04-26 01:10:16 2024-04-26 02:05:16    picked
9          9 2024-04-27 05:37:32 2024-04-27 05:52:32 2024-04-27 06:40:32 